In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import tszip

In [2]:
data_dir = Path("../data")
recomb_file = data_dir / "recombinants.csv"
recomb_df = pd.read_csv(recomb_file)
recomb_df.head(5)

,recombinant,descendants,sample,sample_id,sample_pango,interval_left,interval_right,num_mutations,date_added,group_id,...,Viridian_amplicon_scheme,Platform,Artic_primer_version,break_near_missing_run_12_7,net_min_supporting_loci_lft,net_min_supporting_loci_rgt,net_min_supporting_loci_lft_rgt_ge_4,parent_left_scorpio,parent_right_scorpio,sample_scorpio
0,1530,1,1529,ERR4437465,B.1.157,8783,13617,2,2020-03-22,052b938d3189a1c873abd3ffd894e4c6,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,5,True,.,.,.
1,22500,1,22499,ERR4638271,B.1.1.217,26528,26714,2,2020-07-23,96ae2d6491a4e2d9432c231e6f2bb7a0,...,COVID-ARTIC-V3,ILLUMINA,3,False,11,2,False,.,.,.
2,26465,54,26464,ERR4615866,B.1.221,15325,21855,1,2020-08-24,dccf6a6ff6f85d67cdd5a7e5c0e5d758,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,9,True,.,.,.
3,27003,3,27002,ERR4671078,B.1.160,22993,25563,2,2020-08-26,0c768f9bb44f8c5d4daf59582c6c9a16,...,COVID-ARTIC-V3,OXFORD_NANOPORE,3,False,13,10,True,.,.,.
4,28379,3,28378,SRR21719160,B.1.426,6542,9515,0,2020-09-02,d856a9a4583881e765e24caeb4533948,...,COVID-ARTIC-V3,ILLUMINA,.,False,4,18,True,.,.,.


In [3]:
ts_file = data_dir / "v1-beta1_2023-02-21.pp.md.bpshift.ts.dated.il.tsz"
ts = tszip.decompress(ts_file)

In [4]:
# https://github.com/jeromekelleher/sc2ts/blob/3d0f67b0a284dd76dd60a5aa6788dc01278e07ef/sc2ts/info.py#L28
def convert_date(ts, time_array):
    time_zero_date = ts.metadata["sc2ts"]["date"]
    time_zero_date = np.array([time_zero_date], dtype="datetime64[D]")[0]
    return time_zero_date - time_array.astype("timedelta64[D]")


times_tsdate = ts.nodes_time[recomb_df.recombinant]
dates_tsdate = convert_date(ts, time_array=times_tsdate)

In [5]:
recomb_df.assign(
    time_tsdate=times_tsdate,
    date_tsdate=dates_tsdate,
).to_csv(recomb_file, index=False)